In [4]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras


In [5]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()


In [6]:
print(shakespeare_text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [8]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [9]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [10]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [11]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text]))-1

In [12]:
train_size = dataset_size * 90 //100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [13]:
n_steps=100
window_length = n_steps+1
dataset = dataset.window(window_length,shift=1,drop_remainder=True)

In [14]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [15]:
batch_size = 32
dataset = dataset.shuffle(1000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, :-1]))

In [16]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch,depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=20)

Epoch 1/20
  23676/Unknown - 9764s 412ms/step - loss: 0.0370

In [ ]:
def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts))-1
  return tf.one_hot(X, max_id)
  

In [ ]:

X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

In [ ]:
def next_char(text, tempreature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba)/tempreature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
  return tokenizer.sequences_to_text(char_id.numpy())[0]

In [ ]:
def complete_text(text, n_chars=50, tempreature=1):
  for _ in range(n_chars):
    text += next_char(text, tempreature)
    return text

In [ ]:
print(complete_text("t", tempreature=0.2))